In [4]:
import tensorflow as tf
from tensorflow import keras

- 텐플1에서는 그래프가 텐서플로의 핵심 API였음
- 텐플2에서는 상대적으로 그 중요도가 낮아지고 코드 작성 난이도가 줄어들음

In [5]:
def cube(x):
    return x ** 3

In [6]:
cube(2)

8

In [7]:
cube(tf.constant(2.))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

- 위에서 만든 건 파이썬 함수
- 이걸 tf.function()을 이용하면 텐서플로 함수로 변경 가능

In [8]:
tf_cube = tf.function(cube)
tf_cube

- 이 텐서플로 함수는 파이썬 함수처럼 활용하지만 입력값으로 굳이 tensor를 넣지않아도, 모든  결과를 텐서로 반환하게 됨

In [10]:
tf_cube(2.)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [11]:
tf_cube(tf.constant(2.))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

- 내부적으로 tf.function()는 cube함수에서 수행되는 계산을 분석하고 동일 작업을 수행하는 **계산그래프**를 만드는 것
    - *다른 방식으로는 전에도 나왔던 tf.function 데코레이터가 있는데 보통 이걸 더 많이 사용함*

In [12]:
@tf.function
def tf_cube(x):
    return x**3

In [13]:
tf_cube(2.)

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

- python_function 어트리뷰트를 사용하면 다시 파이썬 함수의 꼴로 사용 가능

In [14]:
tf_cube.python_function(2.)

8.0

- 보통 텐서플로는 사용하지 않는 노드를 제거하고 표현을 단순화하여 그래프를 최적화하고 연산을 최적화함
    - 따라서 일반적으로 파이썬 함수보다 훨씬 빠르게 실행됨(특히 함수가 복잡할 때)
- 그리고 사용자 정의 손실, 지표, 층 등을 사용할때는 케라스가 알아서 이를 텐서플로 함수로 변환함

- 기본적으로 텐서플로 함수는 호출에 사용되는 입력크기와 데이터에 맞춰 매번 새로운 그래프를 생성
    - ex) tf_cube(tf.constant(10))으로 호출 시, [ ]크기의 int32텐서에 맞는 그래프가 생성됨
        - 그 다음 tf_cube(tf.constant(20))을 호출하면 위에서 만들어진 동일한 그래프가 재사용됨
        - 하지만 tf.constant([10, 20])을 호출하면 입력 크기가 바뀌었기 때문에 [2]크기의 int32텐서에 맞는 새로운 그래프를 생성함
    - *이는 인자로 텐서를 사용했을 때만 해당되는 이야기*
        - *만약 파이썬 값을 텐플 함수에 전달하면 각각 그래프를 따로 만들게 됨 -> 효율과 연산 속도에 문제가 생길 수 있음*

### 12.4.1. 오토그래프와 트레이싱
- 텐서플로는 **오토그래프** 기능을 활용하여 파이썬 함수에서 for, while, if, break, continue, return등의 제어문을 모두 찾음
- 그리고 이 모든 제어문을 텐서플로 연산으로 바꾼 업그레이드 버전을 만듦
    - ex) while -> tf.while_loop() // if -> tf.cond() // for -> tf.loop_body()
- 그 다음 텐서플로가 위에서 업그레이드한 함수를 호출하는데, 인자를 직접 전달하는게 아니라 **심볼릭 텐서**를 전달함.
    - 이 과정을 **트레이싱**이라 함
    - 이 텐서는 실제 값은 없고, 이름, 데이터타입, 크기만 가짐
        - 마치 spark에서의 지연연산같은 개념
    - 그래서 이 값이 전달되면, 이 값에 맞게 그래프에 노드만 추가하고 실제 연산은 진행하지 않음

### 12.4.2. 텐서플로 함수 사용법
- 보통 @tf.function 데코레이터만 사용하면 케라스가 나머지를 알아서 처리함
- 다만 유의사항이 몇가지 있음

- 1. 다른 라이브러리를 호출할 경우 트레이싱 과정에서 실행이됨. 따라서 트레이싱 과정에서 코드가 실행되는걸 원치 않으면, np.sum()대신 tf.reduce_, 내장함수 sorted()대신, tf.sort()와 같이 사용해야함
    - tf.py_function으로 처리할 수 있지만, 이러면 속도도 떨어지고, 사용환경에 python이 설치되어있어야만 함
- 2. 다른 파이썬 함수나 텐서플로 함수를 호출할 수 있지만, 이 함수들의 연산을 감지할 수 있음
    - 이런 함수들은 tf.fucntion데코레이터를 적용하지 않아도됨
- 3. 텐서플로 변수를 만든다면, 처음 호출될 때만 수행해야하며, 일반적으로 텐서플로 함수 밖에서 변수를 생성하는 것이 좋음
    - 예를 들면 사용자 정의 층의 build()메서드
    - 변수에 새 값을 할당하고자 한다면, = 연산자 대신 assign()메서드를 이용하는 것이 좋음
- 4. 파이썬 함수의 소스코드는 텐서플로에서 사용 가능해야함
- 5. for문의 경우 텐서나 데이터셋을 순회하는 for문만 감지함
    - 그러므로 for i in range(x)대신, for i in tf.range(x)로 사용해야함
        - 그래야 그래프에 표현됨
    - 하지만 for i in range(x)형식으로 적을 경우 트레이싱단계에서 실행할 수 있음(그래프에 반영X)
- 6. 성능을 생각하면 반복은 최대한 삼가고, 가능한한 벡터화하여 표현하는 것이 필요함